# Sea ice diagnostics for different CMIP6 experiments

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

plt.style.use("seaborn-v0_8-notebook")

## Set parameters

In [ ]:
# Time
year_start = 1985
year_stop = 1986

# Sea Ice Concentration Threshold
sic_threshold = 30

# Models
models = [
    "access_cm2",
    "access_esm1_5",
]

# Chunks for download
chunks = {"year": 1}

## Define request

In [ ]:
common_request = {
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
}

request_era = (
    "reanalysis-era5-single-levels-monthly-means",
    {
        "product_type": "monthly_averaged_reanalysis",
        "format": "netcdf",
        "time": "00:00",
        "variable": "sea_ice_cover",
        **common_request,
    },
)

request_sim = (
    "projections-cmip6",
    {
        "format": "zip",
        "temporal_resolution": "monthly",
        "experiment": "historical",
        "variable": "sea_ice_area_percentage_on_ocean_grid",
        **common_request,
    },
)

## Define function to cache

In [ ]:
def get_sea_ice_diagnostics(ds, sic_threshold, request_model):
    # Get request and model
    collection_id, request = request_model
    model = request.get("model", "ERA5")

    # Same time for all datasets
    ds["time"] = pd.to_datetime(ds["time"].dt.strftime("%Y-%m-01"))

    siconc = ds["siconc"]
    if model == "ERA5":
        # Compute area
        areacello = diagnostics.grid_cell_area(ds)
        siconc *= 100
    else:
        # Request area
        request = {k: v for k, v in request.items() if k not in ("year", "month")}
        request["temporal_resolution"] = "fixed"
        request["variable"] = "grid_cell_area_for_ocean_variables"
        areacello = download.download_and_transform(collection_id, request)["areacello"]
        siconc = ds["siconc"]

    datasets = []
    for region in ("Arctic", "Antartica"):
        mask = ds["latitude"] > 0 if region == "Arctic" else ds["latitude"] < 0
        region_areacello = areacello.where(mask, 0)
        siextent = region_areacello.where(siconc > sic_threshold)
        siarea = region_areacello * siconc
        ds_region = xr.merge(
            [siextent.rename("siextent"), siarea.rename("siarea")]
        ).expand_dims(region=[region])
        datasets.append(ds_region)
    ds = xr.merge(datasets)

    # Sum values
    ds = ds.sum(set(ds.dims) - {"time", "region"})

    # Add CF attributes
    units = areacello.units
    for var in ds.data_vars:
        ds[var].attrs = {
            "standard_name": var.replace("si", "sea_ice_", 1),
            "units": units,
            "long_name": var.replace("si", "Sea ice ", 1),
        }

    return ds.expand_dims(model=[model])

## Download and combine dataset

In [ ]:
datasets = []
for model in ["ERA5"] + models:
    print(f"Downloading and processing {model}")
    if model == "ERA5":
        request_model = request_era
    else:
        request_model = request_sim
        request_model[1]["model"] = model
    ds = download.download_and_transform(
        *request_model,
        chunks=chunks,
        transform_func=get_sea_ice_diagnostics,
        transform_func_kwargs={
            "sic_threshold": sic_threshold,
            "request_model": request_model,
        },
    )
    datasets.append(ds)
ds = xr.concat(datasets, "model")

## Plot variables

In [ ]:
for da in ds.data_vars.values():
    da.plot(col="region", hue="model")
    plt.show()